# SDF Editing Step-By-Step

## Loading the Mesh

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import trimesh

mesh  = trimesh.load(r"../data\bookcase.glb", force='mesh')
#mesh  = trimesh.load(r"../data\completionchair\CompletionChair.obj")
#mesh.show()

## Creating an SDF

In [ ]:
from context import generationtools as gnt
size = 96
sdf,sdfMesh = gnt.mesh_to_sdf_tensor(gnt.as_mesh(mesh), size, recenter=True, scaledownFactor=8)
gnt.sdf_to_mesh(sdf, 1/size).show() # for visualisation, convert to marching cubes mesh

## Creating a colored voxelgrid

In [63]:
voxelMesh, colorGrid, voxelScale = gnt.mesh_to_voxelgrid_trimesh(mesh, resolution = size)

In [ ]:
voxelMesh.as_boxes(colors=colorGrid).show()
#voxelMesh.marching_cubes.show()

## Get colors

In [ ]:
# Generate voxel grid coordinates
voxel_grid = gnt.create_voxel_grid(size, True)
print(voxel_grid.shape)

In [ ]:
# Get the color for each point in the voxel grid
colors = gnt.get_point_colors_trimesh(sdfMesh, voxel_grid.reshape(-1,3))
print(colors.shape)
colors = colors.reshape(size,size,size,4)

### Plot the colors where the SDF distance is < 0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# Plot the voxel grid
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
filled = np.where(sdf<0,1,0)
ax.voxels(filled, facecolors=colors/255, edgecolor='k', linewidth=0.2)

# Set labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=30, azim=30)
# Show plot
plt.show()

## Global non-uniform scaling

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

#pick a slice
oldRange = sdf.shape[0]
newRange = oldRange * 2
# Original and new coordinates along the m-axis
original_m = np.linspace(0, 1, oldRange)  # Normalized original m-axis (0 to 1)
new_m_coords = np.linspace(0, 1, newRange)  # Normalized new m-axis (0 to 1)

# Interpolate along the m-axis for each n x n slice
interpolated_array = np.empty((newRange, sdf.shape[1], sdf.shape[2]))
for i in range( sdf.shape[1]):
    for j in range(sdf.shape[2]):
        # Interpolate along the m-axis for each (i, j) point
        interp_func = interp1d(original_m, sdf[:, i, j], kind='linear', bounds_error=False, fill_value="extrapolate")
        interpolated_array[:, i, j] = interp_func(new_m_coords)

# Check the result
print("Original shape: \n", sdf.shape)
print("Interpolated shape: \n", interpolated_array.shape)

In [ ]:
new_mesh = gnt.sdf_to_mesh(interpolated_array, 1/size)
new_mesh.show()

## Defining The range

In [ ]:
#Set the positions and axis for the planes
startEndRange = np.array([0.5,0.65])  # Positions along the chosen axis normalised from 0 to 1
axis = 2  # Axis for the planes ('x', 'y', or 'z')


# Create the planes
planes = [gnt.create_transparent_plane(pos, axis=axis, size = 1) for pos in startEndRange]

# Create a scene and add the mesh and planes
scene = trimesh.Scene()
scene.add_geometry(gnt.sdf_to_mesh(sdf, 1/size))
for plane in planes:
    scene.add_geometry(plane)

# Show the scene
scene.show()

## Range Interpolating

In [ ]:
interpolated_sdf = gnt.interpolate_value_range(sdf,2, np.floor(startEndRange*96).astype(int), 96)
#interpolated_colors = interpolate_range(colors, [10,20], 42)
new_mesh = gnt.sdf_to_mesh(interpolated_sdf, 1/size)
new_mesh.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# Plot the voxel grid
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
filled = np.where(interpolated_sdf<0,1,0)
ax.voxels(filled, facecolors=interpolated_colors/255, edgecolor='k', linewidth=0.2)

# Set labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=30, azim=30)
# Show plot
plt.show()

## Range value repeating
Scaling an object on a specific axis, repeatingt the selected part a number of times

In [ ]:
#pick a slice
repeatRange = np.floor(startEndRange*96).astype(int)
nrOfRepeats = 10
# Create a new sdf with the repeated ranges
result_array = gnt.repeat_value_range(sdf, axis, repeatRange, nrOfRepeats)

new_mesh = gnt.sdf_to_mesh(result_array, 1/size)
new_mesh.show()

In [ ]:
import numpy as np
from skimage.measure import marching_cubes
import trimesh


# Generate the mesh using marching_cubes
vertices, faces, _, _ = marching_cubes(sdf, level=0)

# Interpolate vertex colors
vertex_colors = gnt.interpolate_vertex_colors(vertices, colors)

# Create a Trimesh object with vertex colors
mesh = trimesh.Trimesh(vertices=vertices, faces=faces, vertex_colors=vertex_colors)

# Visualize the mesh
mesh.show()

# Full Workflow

## Data Preparation

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import trimesh
from context import generationtools as gnt

meshPath = r"../data\completionchair\CompletionChair.obj"
voxelResolution = 64 # The resolution of the voxelgrid

# load the mesh
mesh  = trimesh.load(meshPath)
# convert to SDF
sdf,normalized_mesh = gnt.mesh_to_sdf_tensor(gnt.as_mesh(mesh), voxelResolution, recenter=True, scaledownFactor=0.85)
# Get the colors
voxel_grid = gnt.create_voxel_grid(voxelResolution, True)
colors = gnt.get_point_colors_trimesh(normalized_mesh, voxel_grid.reshape(-1,3)).reshape(voxelResolution,voxelResolution,voxelResolution,4)



### Sampling Check *OPTIONAL*

In [ ]:
# OPTIONAL Evaluate the colors in the SDF mesh
marching_mesh = gnt.sdf_to_mesh(sdf, 1/voxelResolution, True)
# Interpolate vertex colors
vertex_colors = gnt.interpolate_vertex_colors(marching_mesh.vertices, colors)
# Create a Trimesh object with vertex colors
mesh = trimesh.Trimesh(vertices=marching_mesh.vertices, faces=marching_mesh.faces, vertex_colors=vertex_colors[:,:3])
# Visualize the mesh
mesh.show()

## Defining the SelectionPlanes

In [ ]:
startEndRange = np.array([0.45,0.55]) # Positions along the chosen axis normalised from 0 to 1
axis = 0 # Axis for the planes ('x:0', 'y:1', or 'z:2')
# Create the planes
planes = [gnt.create_transparent_plane(pos, axis=axis, size = 1) for pos in startEndRange]
# Create a scene and add the mesh and planes
scene = trimesh.Scene()
scene.add_geometry(gnt.sdf_to_mesh(sdf, 1/voxelResolution, center = True))
scene.add_geometry(sdfMesh)
for plane in planes:
    scene.add_geometry(plane)
print(sdf.shape)
# Show the scene
scene.show()

## Scaling the selection

In [ ]:
newScaleFactor = 10 # factor to which to scale the selected zone can be any positive number
interpolated_sdf = gnt.interpolate_value_range(
    sdf,
    axis, 
    np.floor(startEndRange*voxelResolution).astype(int), 
    np.floor((startEndRange[1]-startEndRange[0]) * voxelResolution * newScaleFactor).astype(int)
    )
# Interpolate the colors
interpolated_colors = gnt.interpolate_value_range(
    colors,
    axis, 
    np.floor(startEndRange*voxelResolution).astype(int), 
    np.floor((startEndRange[1]-startEndRange[0]) * voxelResolution * newScaleFactor).astype(int)
    )
interpolated_mesh = gnt.create_colored_mesh_from_sdf_and_colors(interpolated_sdf, interpolated_colors)
interpolated_mesh.show()

## Repeating the selection

In [ ]:
nrOfRepeats = 10
# Create a new sdf with the repeated ranges
repeated_sdf = gnt.repeat_value_range(sdf, axis, np.floor(startEndRange*voxelResolution).astype(int), nrOfRepeats)
repeated_colors = gnt.repeat_value_range(colors, axis, np.floor(startEndRange*voxelResolution).astype(int), nrOfRepeats)
repeated_mesh = gnt.create_colored_mesh_from_sdf_and_colors(repeated_sdf, repeated_colors)
repeated_mesh.show()